### Read caffe weights and save them in numpy

In [1]:
# code to convert MSGNET from caffe to npy
import caffe
import argparse
import numpy as np
from collections import OrderedDict

In [2]:
# parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument('--proto-path', type=str, default="./models/NYUV2/deploy.prototxt")
# parser.add_argument('--caffemodel-path', type=str, default="./models/NYUV2/cvpr_nyuv2.caffemodel")
# parser.add_argument('--output-path', type=str, default="./caffe_params.npy")
proto_path = "./models/NYUV2/deploy.prototxt"
caffemodel_path = "./models/NYUV2/cvpr_nyuv2.caffemodel"
npy_path = "./caffe_params_nyuv2.npy"

In [3]:
# args = parser.parse_args()
net = caffe.Net(proto_path, caffemodel_path, caffe.TRAIN)

layers = OrderedDict()
for name, params in net.params.items():
    layers[name] = {}
    # probably only works for DORN's caffe model...
    if "bn" in name: # batchnorm
        print("{} left: {}".format(name, len(params) - 4))
        layers[name]["scale"] = params[0].data
        layers[name]["shift"] = params[1].data
        layers[name]["mean"] = params[2].data
        layers[name]["var"] = params[3].data
    elif "ip" in name: # inner product (i.e. fully connected)
        print("{} left: {}".format(name, len(params) - 2))
        layers[name]["weight"] = params[0].data
        layers[name]["bias"] = params[1].data
    else: # convolution
        layers[name]["weight"] = params[0].data
        if len(params) == 2:
            print("{} left: {}".format(name, len(params) - 2))
            layers[name]["bias"] = params[1].data
        else:
            print("{} left: {}".format(name, len(params) - 1))
# Save layers to file
with open(npy_path, "w") as f:
    print("saving to {}".format(npy_path))
    np.save(f, layers)

conv1_1_3x3_s2 left: 0
conv1_1_3x3_s2/bn left: 0
conv1_2_3x3 left: 0
conv1_2_3x3/bn left: 0
conv1_3_3x3 left: 0
conv1_3_3x3/bn left: 0
conv2_1_1x1_reduce left: 0
conv2_1_1x1_reduce/bn left: 0
conv2_1_3x3 left: 0
conv2_1_3x3/bn left: 0
conv2_1_1x1_increase left: 0
conv2_1_1x1_increase/bn left: 0
conv2_1_1x1_proj left: 0
conv2_1_1x1_proj/bn left: 0
conv2_2_1x1_reduce left: 0
conv2_2_1x1_reduce/bn left: 0
conv2_2_3x3 left: 0
conv2_2_3x3/bn left: 0
conv2_2_1x1_increase left: 0
conv2_2_1x1_increase/bn left: 0
conv2_3_1x1_reduce left: 0
conv2_3_1x1_reduce/bn left: 0
conv2_3_3x3 left: 0
conv2_3_3x3/bn left: 0
conv2_3_1x1_increase left: 0
conv2_3_1x1_increase/bn left: 0
conv3_1_1x1_reduce left: 0
conv3_1_1x1_reduce/bn left: 0
conv3_1_3x3 left: 0
conv3_1_3x3/bn left: 0
conv3_1_1x1_increase left: 0
conv3_1_1x1_increase/bn left: 0
conv3_1_1x1_proj left: 0
conv3_1_1x1_proj/bn left: 0
conv3_2_1x1_reduce left: 0
conv3_2_1x1_reduce/bn left: 0
conv3_2_3x3 left: 0
conv3_2_3x3/bn left: 0
conv3_2_1x1_inc

### Load numpy file and pytorch network

In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
npy_path = "./caffe_params_nyuv2.npy"
layers = np.load(npy_path)
layers = layers[()] # Weird indexing thing
# print(layers['ip1/depth']["weight"].shape)
# print(layers["conv1_1_3x3_s2/bn"].keys())
new_layers = {}
for key in layers:
    new_key = key.replace("/", "_")
    new_layers[new_key] = layers[key]
layers = new_layers

In [3]:
# Specific to DORN
state_dict = {}
for layer_name in layers:
    if "bn" in layer_name: # Batchnorm
        state_dict[layer_name + ".weight"] = torch.from_numpy(layers[layer_name]["scale"].flatten())
        state_dict[layer_name + ".bias"] = torch.from_numpy(layers[layer_name]["shift"].flatten())
        state_dict[layer_name + ".running_mean"] = torch.from_numpy(layers[layer_name]["mean"].flatten())
        state_dict[layer_name + ".running_var"] = torch.from_numpy(layers[layer_name]["var"].flatten())
    elif "ip" in layer_name: # inner product
        state_dict[layer_name + ".weight"] = torch.from_numpy(layers[layer_name]["weight"])
        state_dict[layer_name + ".bias"] = torch.from_numpy(layers[layer_name]["bias"])
    else:
        state_dict[layer_name + ".weight"] = torch.from_numpy(layers[layer_name]["weight"])
        if len(layers[layer_name]) == 2:
            state_dict[layer_name + ".bias"] = torch.from_numpy(layers[layer_name]["bias"])
#             print(layer_name, layers[layer_name]["weight"].shape)
#             print(layers[layer_name]["bias"].shape)
        else:
            pass
#             print(layer_name, layers[layer_name]["weight"].shape)
#             state_dict[layer_name + ".bias"] = torch.zeros(layers[layer_name]["weight"].shape[0])

In [4]:
from DORN_pytorch import DORN_nyu

In [5]:
net = DORN_nyu()

In [6]:
net.load_state_dict(state_dict)

In [8]:
ordinal_decode_layer

DORN_nyu(
  (conv1_1_3x3_s2): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (conv1_1_3x3_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.95, affine=True, track_running_stats=True)
  (conv1_1_3x3_s2_relu): ReLU(inplace)
  (conv1_2_3x3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv1_2_3x3_bn): BatchNorm2d(64, eps=1e-05, momentum=0.95, affine=True, track_running_stats=True)
  (conv1_2_3x3_relu): ReLU(inplace)
  (conv1_3_3x3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv1_3_3x3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.95, affine=True, track_running_stats=True)
  (conv1_3_3x3_relu): ReLU(inplace)
  (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2_1_1x1_reduce): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (conv2_1_1x1_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.95, affine=True, track_running_stats=Tr